In [2]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

C:\Users\Ashish\AppData\Local\Temp\ipykernel_8024\1217278582.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
df=pd.read_csv('Real_Combine(1).csv')

In [4]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [5]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64

In [6]:
null_rows = df[df.isnull().any(axis=1)]
null_rows

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
184,14.3,19.2,10.9,1020.5,91.0,1.6,4.8,11.1,NaN


In [7]:
df.dropna(subset=['PM 2.5'],inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1092 entries, 0 to 1092
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   T       1092 non-null   float64
 1   TM      1092 non-null   float64
 2   Tm      1092 non-null   float64
 3   SLP     1092 non-null   float64
 4   H       1092 non-null   float64
 5   VV      1092 non-null   float64
 6   V       1092 non-null   float64
 7   VM      1092 non-null   float64
 8   PM 2.5  1092 non-null   float64
dtypes: float64(9)
memory usage: 85.3 KB


In [9]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

In [10]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] #

In [11]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [12]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=5,
    directory='project',
    project_name='Air Quality Index',
    max_consecutive_failed_trials=10)

Reloading Tuner from project\Air Quality Index\tuner0.json


In [13]:
tuner.search_space_summary()

Search space summary
Default search space size: 19
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [15]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

In [16]:
tuner.results_summary()

Results summary
Results in project\Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 4 summary
Hyperparameters:
num_layers: 8
units_0: 128
units_1: 320
learning_rate: 0.001
units_2: 192
units_3: 256
units_4: 352
units_5: 32
units_6: 128
units_7: 160
units_8: 416
units_9: 32
units_10: 288
units_11: 64
units_12: 416
units_13: 32
units_14: 288
units_15: 224
units_16: 320
Score: 63.88443120320638

Trial 3 summary
Hyperparameters:
num_layers: 5
units_0: 32
units_1: 384
learning_rate: 0.0001
units_2: 128
units_3: 384
units_4: 128
units_5: 224
units_6: 160
units_7: 352
units_8: 416
units_9: 320
units_10: 192
units_11: 128
units_12: 448
units_13: 320
units_14: 448
units_15: 448
units_16: 320
Score: 64.70364634195964

Trial 2 summary
Hyperparameters:
num_layers: 16
units_0: 256
units_1: 448
learning_rate: 0.001
units_2: 512
units_3: 64
units_4: 224
units_5: 320
units_6: 160
units_7: 128
units_8: 64
units_9: 32
units_10: 448
units_11: 128
